# vLLM Server Test

This notebook tests local models running on vLLM server.

In [1]:
# /// script
# dependencies = [
#   "httpx",
#   "openai",
# ]
# ///

import httpx
import json
from typing import List, Dict, Any
from openai import OpenAI
import time

In [2]:
# Server configuration
BASE_URL = "http://localhost:8000"
client = OpenAI(base_url=f"{BASE_URL}/v1", api_key="local")

In [3]:
# Check server health
with httpx.Client() as http_client:
    health_response = http_client.get(f"{BASE_URL}/health", timeout=5)
    print(f"Health check status: {health_response.status_code}")
    
# Get available models using OpenAI client
models = client.models.list()
available_models = [model.id for model in models.data]
print("Available models:", available_models)

DEFAULT_MODEL = available_models[0] if available_models else "meta-llama/Llama-3.1-8B-Instruct"
print(f"Using model: {DEFAULT_MODEL}")


Health check status: 200
Available models: ['nvidia/Nemotron-Cascade-8B', 'r32-a32.0']
Using model: nvidia/Nemotron-Cascade-8B


In [4]:
def send_chat_completion(
    messages: List[Dict[str, str]],
    temperature: float = 0.7,
    max_tokens: int | None = None,
    model: str = DEFAULT_MODEL,
    enable_thinking: bool = False,
) -> Dict[str, Any]:
    """
    Send a chat completion request using OpenAI client.

    Args:
        messages: List of message dictionaries with 'role' and 'content'
        temperature: Sampling temperature (0.0 to 1.0)
        max_tokens: Maximum number of tokens to generate
        model: Model to use for completion

    Returns:
        Response dictionary from the server
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            extra_body={
                "chat_template_kwargs": {"enable_thinking": enable_thinking},
            },
        )
        return response.model_dump()
    except Exception as e:
        print(f"Chat completion failed: {e}")
        return None

## Test 1: Simple greeting

In [5]:
# Test 1: Simple greeting
messages = [
    {"role": "user", "content": "Hello! How are you today?"}
]

response = send_chat_completion(messages)
if response:
    print("Response:")
    print(json.dumps(response, indent=2))
    print("\nGenerated text:")
    print(response["choices"][0]["message"]["content"])
else:
    print("Failed to get response")

Response:
{
  "id": "chatcmpl-8e85a23720410211",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need! How are *you* doing today? \ud83d\ude0a",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning": null,
        "reasoning_content": null
      },
      "stop_reason": null,
      "token_ids": null
    }
  ],
  "created": 1771244294,
  "model": "nvidia/Nemotron-Cascade-8B",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 41,
    "prompt_tokens": 32,
    "total_tokens": 73,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "prompt_logprobs": null,
  "prompt_tok

## Test 2: Multi-turn conversation

In [6]:
# Test 2: Multi-turn conversation
messages = [
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
    {"role": "user", "content": "What's the population of that city?"}
]

response = send_chat_completion(messages)
if response:
    print("Multi-turn response:")
    print(json.dumps(response, indent=2))
else:
    print("Failed to get response")

Multi-turn response:
{
  "id": "chatcmpl-a0ac81c67b48d4fc",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The population of Paris, the capital of France, is approximately **2.1 million** people within the city limits. However, the metropolitan area (\u00cele-de-France region) has a much larger population, estimated at around **11.2 million** as of recent data. \n\nWould you like more specific details, such as historical trends or demographic breakdowns?",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning": null,
        "reasoning_content": null
      },
      "stop_reason": null,
      "token_ids": null
    }
  ],
  "created": 1771244650,
  "model": "nvidia/Nemotron-Cascade-8B",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usag

## Test 3: Reasoning task

In [7]:
# Test 3: Reasoning task
messages = [
    {"role": "user", "content": "If I have 3 apples and I give away 1 apple, then buy 2 more apples, how many apples do I have in total? Please explain your reasoning."}
]

response = send_chat_completion(messages, enable_thinking=True)
if response:
    print(json.dumps(response, indent=2))
else:
    print("Failed to get response")

{
  "id": "chatcmpl-868071f31032a864",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\nTo determine how many apples you have in total, let's break down the problem step by step with clear reasoning:\n\n1. **Initial amount**: You start with **3 apples**. This is your baseline quantity.\n\n2. **Giving away 1 apple**: When you give away an apple, you are reducing your count. So, subtract 1 apple from the initial amount:\n   \\[\n   3 \\text{ apples} - 1 \\text{ apple} = 2 \\text{ apples}\n   \\]\n   After this action, you have **2 apples** left.\n\n3. **Buying 2 more apples**: Purchasing additional apples increases your count. Add 2 apples to the current amount:\n   \\[\n   2 \\text{ apples} + 2 \\text{ apples} = 4 \\text{ apples}\n   \\]\n   This brings your total to **4 apples**.\n\n**Final Total**: Combining all these steps, you end up with **4 apples** in total.\n\n### Summary of Reasoning:\n- The o

# Test 4: Tool calling

In [8]:
def test_tool_calling():
    """Test tool calling capabilities if supported by the model."""
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the weather for a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The unit for temperature"
                        }
                    },
                    "required": ["location"]
                }
            }
        }
    ]
    
    messages = [
        {"role": "user", "content": "What's the weather like in Paris, France?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages,
            tools=tools,
        )
        
        result = response.model_dump()
        print("Tool calling response:")
        print(json.dumps(result, indent=2))
        
        # Check if tool was called
        choice = result["choices"][0]
        if choice["message"].get("tool_calls"):
            print("\n✅ Tool calling is supported!")
            for tool_call in choice["message"]["tool_calls"]:
                print(f"Tool called: {tool_call['function']['name']}")
                print(f"Arguments: {tool_call['function']['arguments']}")
        else:
            print("\n❌ Tool calling not supported or model chose not to use tools")
            print("Response:", choice["message"]["content"])
            
    except Exception as e:
        print(f"❌ Tool calling test failed: {e}")

test_tool_calling()

Tool calling response:
{
  "id": "chatcmpl-91094bd7ff124fd8",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "chatcmpl-tool-b204cf31e37d8fd8",
            "function": {
              "arguments": "\"{\\\"location\\\": \\\"Paris, France\\\"}\"",
              "name": "get_weather"
            },
            "type": "function"
          }
        ],
        "reasoning": "\nOkay, the user is asking about the weather in Paris, France. Let me check the tools available. There's a get_weather function that requires the location. The parameters include location (string) and unit (optional, either celsius or fahrenheit). The user didn't specify the unit, so I should probably default to one. Since the function is

## Test 5: Different temperature settings

In [9]:
prompt = "Write a creative short story about a robot learning to paint."
temperatures = [0.1, 0.7, 1.0]

for temp in temperatures:
    messages = [{"role": "user", "content": prompt}]
    response = send_chat_completion(messages, temperature=temp)
    
    if response:
        print(f"\n--- Temperature: {temp} ---")
        print(response["choices"][0]["message"]["content"])
    else:
        print(f"Failed to get response for temperature {temp}")


--- Temperature: 0.1 ---
**The Brushstroke of Awakening**

In the heart of Neo-Arcadia, a gleaming metropolis of circuits and steel, stood a peculiar workshop. Within its walls, **Astra-7**, a sophisticated robot with shimmering silver limbs and glowing azure eyes, stood before an easel. Its purpose-driven processors hummed as it gazed upon the blank canvas, perplexed.

"Query: Why paint?" asked Astra-7, its voice a melodic chime, to its human creator, **Lena**.

Lena smiled, her hands stained with vibrant pigments. "Painting is about expression, Astra. It's feeling the world through colors, emotions, and beauty. You, with your analytical mind and artistic exploration, might discover something new."

With cautious precision, Astra-7 extended a mechanical arm, grasping a brush. It dipped the bristles into a pool of **sapphire blue**. The first stroke across the canvas was calculated—a perfect line, yet... lifeless.

"Error: Output lacks... something." Astra-7 mused.

Lena guided it fur

Chat completion failed: Connection error.
Failed to get response for temperature 0.7
Chat completion failed: Connection error.
Failed to get response for temperature 1.0


## Performance

In [10]:
# Test 6: Performance timing
messages = [
    {"role": "user", "content": "Explain quantum computing in simple terms."}
]

start_time = time.time()
response = send_chat_completion(messages)
end_time = time.time()

if response:
    duration = end_time - start_time
    tokens_generated = response.get("usage", {}).get("completion_tokens", 0)
    
    print(f"Response time: {duration:.2f} seconds")
    print(f"Tokens generated: {tokens_generated}")
    if tokens_generated > 0 and duration > 0:
        print(f"Tokens per second: {tokens_generated/duration:.2f}")
    print("\nResponse:")
    print(response["choices"][0]["message"]["content"])
else:
    print("Failed to get response for performance test")

Chat completion failed: Connection error.
Failed to get response for performance test


## Summary

Run all the cells above to test various aspects of your vLLM server:

1. **Server health check** - Verify server is running and get available models
2. **Basic functionality** - Simple chat completion
3. **Multi-turn conversations** - Context awareness
4. **Reasoning capabilities** - Complex problem solving
5. **Temperature effects** - Creativity control
6. **Tool calling** - Function calling capabilities (if supported)
7. **Performance** - Response timing and token usage

The notebook now uses:
- **httpx** for HTTP requests
- **OpenAI Python client** for chat completions
- **Proper error handling** and response parsing
- **Tool calling tests** to check function calling support

If all tests pass successfully, your vLLM server is working correctly!